In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#default_exp classify_image

In [4]:
#export
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *

In [5]:
collection_name = 'food'

In [6]:
#export
foods = read_sql('foods')
# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')

In [7]:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'http://127.0.0.1:8181/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return df.reset_index()

In [8]:
import numpy as np
import torch
from torch.nn import CosineSimilarity
cos = CosineSimilarity(dim=1, eps=1e-08)

In [11]:
url = 'https://www.campbellsmeat.com/images/cms/large/142804413214140689200001550_nicknairnsperfectsteakandchipsrecipe.jpeg'
df = search_image(url,head=70)
clip = torch.tensor([np.array(c) for c in df['clip'].values])
best = df.sort_values("score",ascending=False).iloc[0].name
df['score1'] =cos(clip, clip[best])
df['score2'] =cos(clip, clip[df.sort_values("score1",ascending=False).iloc[-1].name])
for i in range(3,10):
    df[f'score{i}'] =cos(clip, clip[df[df.columns[df.columns.str.contains('score')]]         .sum(1).sort_values().index[0]])
df[(df[df.columns[df.columns.str.contains('score')]]==1).any(1)]['description'].drop_duplicates()

KeyboardInterrupt: 

In [14]:
len(foods['category'].unique())

167

In [15]:
foods

,description,category,energy,protein,carb,fat,clip
id,,,,,,,
1097524,"Milk, lactose free, reduced fat (2%)","Milk, reduced fat",50.0,3.35,4.91,1.90,"[0.015040726, 0.011575523, -0.011966778, 0.040..."
1097525,"Milk, lactose free, whole","Milk, whole",60.0,3.28,4.67,3.20,"[0.014815284, -0.004186348, -0.007967781, 0.02..."
1097603,"Yogurt, whole milk, baby food",Baby food: yogurt,61.0,3.47,4.66,3.25,"[0.0027644234, 0.0029128625, 0.022239361, 0.03..."
1097700,"Infant formula, NFS","Formula, ready-to-feed",66.0,1.38,7.12,3.56,"[0.0023559248, 0.017568443, 0.033846956, 0.020..."
1099619,Cabbage with ham hocks,Meat mixed dishes,71.0,7.97,4.50,2.49,"[0.03035846, 0.0039041196, -0.014388906, 0.015..."
...,...,...,...,...,...,...,...
1104568,Cheese as ingredient in sandwiches,Not included in a food category,360.0,20.76,4.32,28.93,"[0.0047567077, 0.03878048, 0.043640774, -0.006..."
1104569,Breakfast meat as ingredient in omelet,Not included in a food category,253.0,20.81,1.61,17.87,"[0.0051773326, 0.019715708, 0.0039956337, -0.0..."
1104570,"Fish fillet, fried as ingredient in sandwiches",Not included in a food category,200.0,18.11,8.18,10.03,"[0.009361401, 0.023042753, 0.011918317, 0.0034..."


In [83]:
df[df.columns.str.contains('score')]

ValueError: Item wrong length 8 instead of 30.

In [ ]:
listing_id_pics['score_to_text'] = search('beautiful interior')
best_to_text = listing_id_pics.sort_values("score_to_text",ascending=False).iloc[0].name
listing_id_pics['score1'] =cos(clip, clip[best_to_text])
selected1 =listing_id_pics.sort_values("score1",ascending=False).iloc[-1].name
listing_id_pics['score2'] =cos(clip, clip[selected1])
selected2 = listing_id_pics[['score1','score2']].sum(1).sort_values().index[0]
listing_id_pics['score3'] =cos(clip, clip[selected2])
selected3 = listing_id_pics[['score1','score2','score3']].sum(1).sort_values().index[0]
listing_id_pics['score4'] =cos(clip, clip[selected3])

In [28]:
scores_to_best = cos(t[0],t)

In [30]:
scores_to_best.min()

tensor(0.5899, dtype=torch.float64)

In [37]:
fartherst = np.argwhere(scores_to_best== scores_to_best.min())[0][0]

In [42]:
fartherst

tensor(7)

In [54]:
n = torch.stack((t[0],t[fartherst]))

In [55]:
t.shape

torch.Size([30, 768])

In [56]:
cos(n,t)

RuntimeError: The size of tensor a (2) must match the size of tensor b (30) at non-singleton dimension 0

In [32]:
scores_to_best

tensor([1.0000, 0.7207, 0.6929, 0.7595, 0.7595, 0.6921, 0.7194, 0.5899, 0.5899,
        0.6968, 0.6792, 0.6705, 0.6206, 0.6885, 0.6584, 0.7086, 0.6448, 0.6886,
        0.6559, 0.6683, 0.6818, 0.6906, 0.7271, 0.7271, 0.6717, 0.6969, 0.6811,
        0.6784, 0.7159, 0.6701], dtype=torch.float64)

In [33]:
r

,id,description,clip,score
0,41,"Cheese, cottage, lowfat, 2% milkfat","[0.029066313, 0.018294085, -0.014341239, 0.003...",0.204797
1,165,"Flour, soy, full-fat","[-0.0004145672, 0.010990089, -0.012770833, 0.0...",0.196780
2,207,"Flour, semolina, fine","[0.004440714, -0.01190605, 0.017707396, 0.0257...",0.195567
3,100,"Cheese, ricotta, whole milk","[0.039640594, 0.027213166, 0.022343373, 0.0154...",0.193412
4,11,"Cheese, ricotta, whole milk","[0.039640594, 0.027213166, 0.022343373, 0.0154...",0.193412
5,168,"Flour, pastry, unenriched, unbleached","[0.011388066, 0.025323315, 7.580683e-05, 0.024...",0.191500
6,205,"Flour, spelt, whole grain","[-0.0024113043, 0.0023404257, -0.00039863735, ...",0.190714
7,192,"Spinach, baby","[0.032259777, -0.029398972, 0.0034496144, -0.0...",0.182451
8,201,"Spinach, baby","[0.032259777, -0.029398972, 0.0034496144, -0.0...",0.182451
9,152,"Flour, wheat, all-purpose, enriched, unbleached","[0.0032659338, 0.025198318, -0.00913277, 0.043...",0.181778


In [107]:
!nbdev_build_lib

Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_classfy_image.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted inference.ipynb.
Converted stats.ipynb.
Converted usda_parsing.ipynb.


In [3]:
from_pickle('image_url')

'https://api.telegram.org/file/bot5203882708:AAG3G_Y2oZSr-rMG2zoffDVtj3d0KkOFSts/photos/file_1040.jpg'